In [55]:
# standard libraries
import time
import pandas as pd
import numpy as np
from sklearn import preprocessing
# import urllib.request
# from pathlib import Path
# from urllib.error import HTTPError
from tqdm.notebook import tqdm 

In [18]:
# Pytorch and Pytorch Lightning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader, random_split
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint


In [19]:
# Visualization and plotting
import umap
import plotly.express as px

# Tensorboard extension
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Data

In [20]:
# Load data set 
rna = pd.read_csv("data/second/rna_scale.csv",index_col=0).T
pro = pd.read_csv("data/second/protein_scale.csv", index_col=0).T
metadata = pd.read_csv("data/second/metadata.csv",index_col=0)

In [21]:
ncells = rna.shape[0]
nfeatures_rna = rna.shape[1]
nfeatures_pro = pro.shape[1]

print("Number of cells:", ncells)
print("Number of genes:", nfeatures_rna)
print("Number of proteins:", nfeatures_pro)

Number of cells: 30672
Number of genes: 2000
Number of proteins: 25


In [22]:
# concat 
assert all(rna.index == pro.index), "RNA and protein data cell barcodes do not match!"
citeseq = pd.concat([rna, pro], axis=1)
print(citeseq.shape)
citeseq.head()

(30672, 2025)


,IGKC,HBA2,HBB,HBA1,IGHA1,IGLC2,JCHAIN,HBM,IGHG1,IGHM,...,CD38,CD4,CD45RA,CD45RO,CD56,CD57,CD69,CD79b,CD8a,HLA.DR
a_AAACCTGAGCTTATCG-1,-0.428204,0.183748,0.406723,-0.488968,1.248214,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,...,1.046025,-0.578292,-0.685804,-0.618250,0.486534,1.084345,0.114363,-0.279477,-0.474619,0.998738
a_AAACCTGAGGTGGGTT-1,-1.047339,-0.601956,-0.985002,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,4.964708,-0.414643,...,-0.665740,1.435716,-1.205478,0.359657,2.607521,-0.612902,-1.146097,0.245467,-0.762164,-0.838409
a_AAACCTGAGTACATGA-1,-1.047339,-0.601956,1.534527,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,...,0.010368,1.745735,0.426769,-1.072402,-0.223553,1.059687,-0.635409,-0.844739,-0.448139,-1.074838
a_AAACCTGCAAACCTAC-1,0.080052,0.828744,0.614775,1.051572,-0.552748,0.713069,-0.324617,-0.181656,-0.200979,-0.414643,...,-0.202340,1.596914,-1.300920,2.089684,-0.637078,-0.166773,-0.541819,-0.646660,-0.663541,-0.589663
a_AAACCTGCAAGGTGTG-1,0.953832,-0.601956,0.794895,-0.488968,1.413335,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,...,1.264582,-0.308044,-1.246741,0.639829,-0.511057,0.300100,1.660178,-0.351614,-0.581021,0.766440


In [23]:
metadata.head()

,orig.ident,nCount_RNA,nFeature_RNA,nCount_ADT,nFeature_ADT,lane,donor,celltype.l1,celltype.l2,RNA.weight,ADT.weight,wsnn_res.2,seurat_clusters
a_AAACCTGAGCTTATCG-1,bmcite,7546,2136,1350,25,HumanHTO4,batch1,Progenitor cells,Prog_RBC,0.487299,0.512701,19,19
a_AAACCTGAGGTGGGTT-1,bmcite,1029,437,2970,25,HumanHTO1,batch1,T cell,gdT,0.245543,0.754457,10,10
a_AAACCTGAGTACATGA-1,bmcite,1111,429,2474,23,HumanHTO5,batch1,T cell,CD4 Naive,0.501680,0.498320,1,1
a_AAACCTGCAAACCTAC-1,bmcite,2741,851,4799,25,HumanHTO3,batch1,T cell,CD4 Memory,0.431308,0.568692,4,4
a_AAACCTGCAAGGTGTG-1,bmcite,2099,843,5434,25,HumanHTO2,batch1,Mono/DC,CD14 Mono,0.572097,0.427903,2,2


In [ ]:
# CITE-seq dataset used by tutorial - 8000+ cord blood mononuclear cells 
# 13 proteins 
# one CSV for gene expression and one for protein expression 

# concat
scale_data = pd.concat([scale_data, protein_data], axis=1, join='inner', sort=False)
scale_data.head()

In [24]:
assert all(citeseq.index == pro.index), "CITE-seq data and metadata cell barcodes do not match!"

# separate CD4 and CD8 in l1
metadata["celltype.l1.5"] = metadata["celltype.l1"].values
metadata.loc[metadata["celltype.l2"].str.startswith("CD4"), "celltype.l1.5"] = "CD4 T"
metadata.loc[metadata["celltype.l2"].str.startswith("CD8"), "celltype.l1.5"] = "CD8 T"
metadata.loc[metadata["celltype.l2"]=="Treg", "celltype.l1.5"] = "CD4 T"
metadata.loc[metadata["celltype.l2"]=="MAIT", "celltype.l1.5"] = "MAIT"
metadata.loc[metadata["celltype.l2"]=="gdT", "celltype.l1.5"] = "gdT"

# convert cell type annoations to integers
le = preprocessing.LabelEncoder()
labels = le.fit_transform(metadata["celltype.l1.5"])

### Pytorch datasets and dataloaders

In [25]:
class TabularDataset(Dataset):
    """Custome dataset for tabular data"""
    def __init__(self, df: pd.DataFrame, labels: np.ndarray):
        self.data = torch.tensor(df.to_numpy(), dtype=torch.float)
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y

In [27]:
dataset = TabularDataset(citeseq, labels)

# train, validation, and test split
train_size = int(ncells*0.7)
val_size = int(ncells*0.15)
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, ncells-train_size-val_size],
                                         generator=torch.Generator().manual_seed(0))


In [28]:
print("Number of cells for training:", len(train_ds))
print("Number of cells for validation:", len(val_ds))
print("Number of cells for test:", len(test_ds))

Number of cells for training: 21470
Number of cells for validation: 4600
Number of cells for test: 4602


In [29]:
# batch size
bs = 256

train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, drop_last=True, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, drop_last=False)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=False, drop_last=False)

In [30]:
x, y = train_dl.dataset[0]
print("Input data:", x)
print("Label:     ", y)

Input data: tensor([ 0.4667, -0.6020,  1.3389,  ..., -0.3434, -0.5349, -0.2583])
Label:      tensor(1.)


### Autoencoder
Encoder compresses input, bottleneck layer stores compressed representation, decoder attempts to reconstruct


Unsupervised deep learning - Autoencoders
CITE-seq (Cellular Indexing of Transcriptomes and Epitopes by sequencing) 
- simultaneously capture RNA and surface protein expression 
This requires computational methods that can effectively integrate single-cell data from both modalities 

In [ ]:
# encode gene and protein data with different encoders since they have different dimensions
# concatenate outputs before passing it through another encoder to generate bottleneck layer 
# We can change the neural network architecture given 
class Autoencoder(nn.Module)

In [31]:
class LinBnDrop(nn.Sequential):
    """Module grouping `BatchNorm1d`, `Dropout` and `Linear` layers, adapted from fastai."""
    
    def __init__(self, n_in, n_out, bn=True, p=0., act=None, lin_first=True):
        layers = [nn.BatchNorm1d(n_out if lin_first else n_in)] if bn else []
        if p != 0: layers.append(nn.Dropout(p))
        lin = [nn.Linear(n_in, n_out, bias=not bn)]
        if act is not None: lin.append(act)
        layers = lin+layers if lin_first else layers+lin
        super().__init__(*layers)

In [32]:
class Encoder(nn.Module):
    """Encoder for CITE-seq data"""
    def __init__(self,
                 nfeatures_rna: int,
                 nfeatures_pro: int,
                 hidden_rna: int,
                 hidden_pro: int,
                 latent_dim: int,
                 p: float = 0):
        super().__init__()
        self.nfeatures_rna = nfeatures_rna
        self.nfeatures_pro = nfeatures_pro
        hidden_dim = hidden_rna + hidden_pro
        
        self.encoder_rna = nn.Sequential(
            LinBnDrop(nfeatures_rna, nfeatures_rna // 2, p=p, act=nn.LeakyReLU()),
            LinBnDrop(nfeatures_rna // 2, hidden_rna, act=nn.LeakyReLU())
        )
        self.encoder_protein = LinBnDrop(nfeatures_pro, hidden_pro, p=p, act=nn.LeakyReLU())
        self.encoder = LinBnDrop(hidden_dim, latent_dim, act=nn.LeakyReLU())

    def forward(self, x):
        x_rna = self.encoder_rna(x[:, :self.nfeatures_rna])
        x_pro = self.encoder_protein(x[:, self.nfeatures_rna:])
        x = torch.cat([x_rna, x_pro], 1)
        return self.encoder(x)

In [33]:
class Decoder(nn.Module):
    """Decoder for CITE-seq data"""
    def __init__(self,
                 nfeatures_rna: int,
                 nfeatures_pro: int,
                 hidden_rna: int,
                 hidden_pro: int,
                 latent_dim: int):
        super().__init__()
        hidden_dim = hidden_rna + hidden_pro
        out_dim = nfeatures_rna + nfeatures_pro
        
        self.decoder = nn.Sequential(
            LinBnDrop(latent_dim, hidden_dim, act=nn.LeakyReLU()),
            LinBnDrop(hidden_dim, out_dim // 2, act=nn.LeakyReLU()),
            LinBnDrop(out_dim // 2, out_dim, bn=False)
            )

    def forward(self, x):
        return self.decoder(x)

In [34]:
class CiteAutoencoder(pl.LightningModule):
    def __init__(self,
                 nfeatures_rna: int,
                 nfeatures_pro: int,
                 hidden_rna: int,
                 hidden_pro: int,
                 latent_dim: int,
                 p: float = 0,
                 lr: float = 0.1):
        """ Autoencoder for citeseq data """
        super().__init__()
        
        # save hyperparameters
        self.save_hyperparameters()
 
        self.encoder = Encoder(nfeatures_rna, nfeatures_pro, hidden_rna, hidden_pro, latent_dim, p)
        self.decoder = Decoder(nfeatures_rna, nfeatures_pro, hidden_rna, hidden_pro, latent_dim)
        
        # example input array for visualizing network graph
        self.example_input_array = torch.zeros(256, nfeatures_rna + nfeatures_pro)

    def forward(self, x):
        # extract latent embeddings
        z = self.encoder(x)
        return z
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}
    
    def _get_reconstruction_loss(self, batch):
        """ Calculate MSE loss for a given batch. """
        x, _ = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        # MSE loss
        loss = F.mse_loss(x_hat, x)
        return loss
    
    def training_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("val_loss", loss)
        
    def test_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("test_loss", loss)

In [41]:
# Path to saved models
from pathlib import Path
CHECKPOINT_PATH = Path("saved_models")
if not CHECKPOINT_PATH.exists():
    CHECKPOINT_PATH.mkdir()
# Use GPU if available, otherwise use cpu instead 
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)


Device: cpu


In [39]:
def train_citeseq(hidden_rna: int = 30, hidden_pro: int = 18,
                  latent_dim: int = 24, p: float = 0.1, lr: float = 0.1):
    trainer = pl.Trainer(default_root_dir=CHECKPOINT_PATH,
                         gpus=1 if "cuda" in str(device) else 0,
                         max_epochs=50,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="val_loss"),
                                    LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True
    trainer.logger._default_hp_metric=None
    
    model = CiteAutoencoder(nfeatures_rna,
                            nfeatures_pro,
                            hidden_rna=hidden_rna,
                            hidden_pro=hidden_pro,
                            latent_dim=latent_dim,
                            p=p,
                            lr=lr)
    trainer.fit(model, train_dl, val_dl)
    
    train_result = trainer.test(model, train_dl, verbose=False)
    val_result = trainer.test(model, val_dl, verbose=False)
    test_result = trainer.test(model, test_dl, verbose=False)
    result = {"train": train_result, "val": val_result, "test": test_result, }
    return model, result

In [43]:
model, result = train_citeseq()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params | In sizes    | Out sizes
--------------------------------------------------------------
0 | encoder | Encoder | 2.0 M  | [256, 2025] | [256, 24]
1 | decoder | Decoder | 2.1 M  | ?           | ?        
--------------------------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.548    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

In [44]:
print(f"Training loss:  {result['train'][0]['test_loss']:.3f}")
print(f"Validation loss:  {result['val'][0]['test_loss']:.3f}")
print(f"Test loss: {result['test'][0]['test_loss']:.3f}")

Training loss:  0.630
Validation loss:  0.637
Test loss: 0.632


### visualisation

In [51]:
%tensorboard --host 0.0.0.0 --port 8000 --logdir saved_models/lightning_logs/version_0

In [53]:
# kill tensorboard process
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')

In [100]:
test_encodings = []
test_labels = []
    
model.eval()
with torch.no_grad():    
    for x, y in tqdm(test_dl, desc="Encoding cells"):
        test_encodings.append(model(x.to(model.device)))
        test_labels += y.to(torch.int).tolist()
        
test_embeds = torch.cat(test_encodings, dim=0).cpu().numpy()
test_labels = le.inverse_transform(test_labels)

Encoding cells:   0%|          | 0/18 [00:00<?, ?it/s]

In [101]:
# run umap for dimensionality reduction and visualization
embeds_umap = umap.UMAP(random_state=0).fit_transform(test_embeds)

In [113]:
plot_df = pd.DataFrame(test_labels.copy())
plot_df["UMAP1"] = embeds_umap[:, 0]
plot_df["UMAP2"] = embeds_umap[:, 1]
fig = px.scatter(plot_df, x="UMAP1", y="UMAP2", color=0)
fig.update_traces(marker=dict(size=4))
fig.show()